# OnSSET GIS-Extraction Notebook

**Conceptualization, Methodology & Original code**: [Babak Khavari](https://github.com/babakkhavari) <br/>
**Review and Updates**: [Babak Khavari](https://github.com/babakkhavari)

This is the GEP-OnSSET GIS extraction notebook. This is supposed to act as a complement to the QGIS plugins available [here](https://github.com/global-electrification-platform/Cluster-based_extraction_OnSSET/tree/master/Plugin). 

The main purpose of this notebook is to facilitate the change of single datasets without running through the entire plugin. Using this notebook the user will be able to change however many datasets needed.

In order to run an OnSSET analysis the following datasets are needed:
* Land Cover
* Elevation 
* Slope
* Global horizontal irradiation
* Travel time
* Wind velocity
* Clusters **(these clusters should include: The name of the study area, the amount of nighttime lights, population, population living in areas with nighttime light and an ID column)**. The clusters can be downloaded from [Energydata.info](https://energydata.info/) or generated directly using the [following plugin](https://github.com/global-electrification-platform/Clustering)

In addition to this there are also some optional datasets that can be used in the analysis:
* Custom Demand - A layer that can be created by the users themselves. For the first round of GEP the methodlogy described [here](https://www.mdpi.com/1996-1073/12/7/1395) has been used.
* Substations
* Transformers
* Mini/Small hydro
* Existing and planned HV-lines
* Existing and planned MV-lines 
* Road network

Below instructions for each cell follows. The cells marked with **(Mandatory)** in the title have to be run.

## Cell 1 - Importing necessary packages (Mandatory)

Packages to be used are imported from the funcs.ipynb.

In [ ]:
from ipynb.fs.full.funcs import *

## Cell 2 - Setting the target coordinate system (Mandatory)

When calculating distances it is important to choose a coordinate system that represents distances correctly in your area of interst. The coordinate system that is given below is the World Mercator, these coordinate system works well for Sub Saharan Africa but the distortions get larger as you move away from the equator.

In order to select your own coordinate system go to [epsg.io](http://epsg.io/) and type in your area of interest, this will give you a list of coordinate systems to choose from. Once you have selected your coordinate system replace the numbers below with the numbers from your coordinate system **(keep the "EPSG" part)**.

**NOTE** When selecting your coordinate system make sure that you select a system with the unit of meters, this is indicated for all systems on [epsg.io](http://epsg.io/)

In [ ]:
crs = 'EPSG:3395'

## Cell 3 - Select the workspace and the administrative boundaries (Mandatory)

Define the workspace. The output layers will populate this folder. It is highly recommended to select an empty folder as your workspace.

For the administrative boundaries you will have to select an **Polygon** layer represeting your area of interest.
        

In [ ]:
messagebox.showinfo('OnSSET extraction', 'Output folder')
workspace = filedialog.askdirectory()

messagebox.showinfo('OnSSET', 'Select the admin boundaries')
admin = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

## Cell 4 - Select the population clusters (Mandatory)

Select the clusters to be used in the analysis

Please also idicate which column is representing the population data as this will be used later. 

In [ ]:
messagebox.showinfo('OnSSET', 'Select the clusters')
clusters = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    
popunit = widgets.Dropdown(options=clusters.head(),
    value=None,
    description='Population:',
    disabled=False)
display(popunit)
x = popunit.value

## Cell 5 - Select the Land Cover map (Raster map)

**If your settlement data already includes land cover data, skip to cell 6. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the land cover map that you wish to use in your analysis. This cell will extract the land cover values in your raster map to your clusters.


In [ ]:
clusters = processing_raster("landcover","majority",clusters)

## Cell 6 - Select the Elevation map (Raster map)

**If your settlement data already includes elevation and slope data, skip to cell 7. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the elevation map that you wish to use in your analysis. This cell will extract the elevation values in your raster map to your clusters. This cell will also generate the a map for the terrain slope. 

In [ ]:
clusters = processing_elevation_and_slope("elevation","mean",clusters, workspace,crs)

## Cell 7 - Select the Global Horizontal Irradiation (GHI) map (Raster map)

**If your settlement data already includes GHI data, skip to cell 8. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the ghi map that you wish to use in your analysis. This cell will extract the ghi values in your raster map to your clusters.

In [ ]:
clusters = processing_raster("ghi","mean",clusters)

## Cell 8 - Select the Travel Time map (Raster map)
 
**If your settlement data already includes travel time data, skip to cell 9. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the travel time map that you wish to use in your analysis. This cell will extract the travel time values in your raster map to your clusters.

In [ ]:
clusters = processing_raster("traveltime","mean",clusters)

## Cell 9 - Select the Wind Velocity map (Raster map)

**If your settlement data already includes wind velocity data, skip to cell 10. Note however that this dataset is mandatory to run the OnSSET analysis**

Select the wind velocity map that you wish to use in your analysis. This cell will extract the wind velocity values in your raster map to your clusters.

In [ ]:
clusters = processing_raster("wind","mean",clusters)

## Cell 10 - Select the Custom Demand map (Raster map) (optional dataset)

Select the custom demand map that you wish to use in your analysis. This is an optional dataset. 

This cell will extract the custom demand values in your raster map to your clusters.

In [ ]:
clusters = processing_raster("customdemand","mean",clusters)

## Cell 11 - Finalizing the raster data


Saving the clusters with extracted rasters.

**NOTE** You have to run this cell if you ran any of the cells 5 through 10. If you did not run any of the mentioned cells skip to cell 12.

In [ ]:
clusters = finalizing_rasters(workspace, clusters, crs)

## Cell 12 - Preparing to run the vector data

**If you are planning on extracting any vector data (substations, transformers, hydro, MV-lines, HV-lines or roads) run this cell**. 

This cell reprojects the settlements to the coordinate system you specified above.

In [ ]:
clusters = preparing_for_vectors(workspace, clusters, crs)

## Cell 13 - Substations (Vector point layer)

**If you do not have substations or wish to keep the ones already in your settlement file, skip to cell 14.**

Determines the distances between each settlement point to the closest substation. 

In [ ]:
clusters = processing_points("Substation", admin, crs, workspace, clusters)

## Cell 14 - Existing high voltage lines (Vector line layer)

**If you do not have existing high voltage lines or wish to keep the ones already in your settlement file, skip to cell 15.**

Determines the distances between each settlement point to the closest existing high voltage line. 

In [ ]:
clusters = processing_lines("Existing_HV", admin, crs, workspace, clusters)

## Cell 15 - Planned high voltage lines (Vector line layer)

**If you do not have planned high voltage lines or wish to keep the ones already in your settlement file, skip to cell 16.**

Determines the distances between each settlement point to the closest planned high voltage line. 

In [ ]:
clusters = processing_lines("Planned_HV", admin, crs, workspace, clusters)

## Cell 16 - Existing medium voltage lines (Vector line layer)

**If you do not have existing medium voltage lines or wish to keep the ones already in your settlement file, skip to cell 17.**

Determines the distances between each settlement point to the closest existing medium voltage line. 

In [ ]:
clusters = processing_lines("Existing_MV", admin, crs, workspace, clusters)

## Cell 17 - Planned medium voltage lines (Vector line layer)

**If you do not have planned medium voltage lines or wish to keep the ones already in your settlement file, skip to cell 18.**

Determines the distances between each settlement point to the closest planned medium voltage line. 

In [ ]:
clusters = processing_lines("Planned_MV", admin, crs, workspace, clusters)

## Cell 18 - Roads (Vector line layer)

**If you do not have roads or wish to keep the ones already in your settlement file, skip to cell 19.**

Determines the distances between each settlement point to the closest road. 

In [ ]:
clusters = processing_lines("Roads", admin, crs, workspace, clusters)

## Cell 19 - Transformers (Vector point layer)

**If you do not have transformers or wish to keep the ones in the already in the settlement file, skip to cell 20** 

Determines the distances between each settlement point to the closest transformer. 

In [ ]:
clusters = processing_points("Transformer", admin, crs, workspace, clusters)

## Cell 20 and 21 - Selecting and processing hydro points (Vector point layer)

**If you do not have new hydro power points skip to cell 22** 

**In Cell 20** Select the hydro point layer you wish to use. It is important to have a column representing the power output for each hydro point in your dataset. After selecting the column you will also have to select the unit (W, kW or MW). 

**In Cell 21** When everything is selected in cell 20, run cell 21 in order to determine the distane to the closest hydro point for each settlement.

In [ ]:
messagebox.showinfo('OnSSET', 'Select the Hydropower map')
hydro=gpd.read_file(filedialog.askopenfilename(title = "Select Hydro map", filetypes = (("shapefile","*.shp"),("all files","*.*"))))

hydropower = widgets.Dropdown(options=hydro.head(),
    value=None,
    description='Hydropower:',
    disabled=False)

display(hydropower)
      
hydrounit = widgets.Dropdown(options=['W', 'kW', 'MW'],
    value='W',
    description='Unit:',
    disabled=False)

display(hydrounit)

In [ ]:
clusters = processing_hydro(admin, crs, workspace, clusters, hydro, hydropower.value, hydrounit.value)

## Cell 22 - Conditioning & Export (Mandatory)

This is the final cell in the extraction. This cell has to be run.

In [ ]:
clusters = conditioning(clusters, workspace, popunit.value)